#### Image Classification: Training and testing of Inception ResNet V2, using CLR for learning rate
The code for implementing Inception ResNet V2 using a form of transfer learning is based on the code by Kwot Sin: 
###### https://kwotsin.github.io/tech/2017/02/11/transfer-learning.html
Using this tutorial, the initial IRNV2 code was created. There have been several alterations in the code, with the most important example changing the learning rate 'model'. For this, Cyclical Learning Rate (CLR) as proposed in the paper by Smith (2017): "Cyclical learning rates for training neural networks".
Other alterations include saving the predicted probabilities of the model, instead of only an accuracy score.

In [18]:
import tensorflow as tf
from tensorflow.contrib.framework.python.ops.variables import get_or_create_global_step
from tensorflow.python.platform import tf_logging as logging
import inception_preprocessing
import inception_daan_preprocessing
import learning_rate_decay_daan
from learning_rate_decay_daan import cyclic_lr
from inception_resnet_v2 import inception_resnet_v2, inception_resnet_v2_arg_scope
import os
import time
slim = tf.contrib.slim
import math
import numpy as np
import matplotlib.pyplot as plt

Starting point: State directories & load labels

In [19]:
#State the labels file and read it
labels_file = 'C:/Users\studentid\Desktop\JADS - Master Thesis\Data\ImagesFinalData/labels.txt'
labels = open(labels_file, 'r')

#State image size
image_size = 299

#Create a dictionary to refer each label to their string name
labels_to_name = {}
for line in labels:
    label, string_name = line.split(':')
    string_name = string_name[:-1] #Remove newline
    labels_to_name[int(label)] = string_name

#Create the file pattern of your TFRecord files so that it could be recognized later on
file_pattern = 'productimages_%s_*.tfrecord'

#Create a dictionary that will help people understand your dataset better. This is required by the Dataset class later.
items_to_descriptions = {
    'image': 'Image of a product in the dataset, out of 275 classes',
    'image_id': 'Unique identifier for each image',
    'label': 'A label that matches the class of the image.'
}

Training & Validation Data

In [7]:
#============== DATASET LOADING ======================
#We now create a function that creates a Dataset class which will give us many TFRecord files to feed in the examples into a queue in parallel.
def get_split(split_name, dataset_dir, file_pattern=file_pattern):
#     '''
#     Obtains the split - training or validation - to create a Dataset class for feeding the examples into a queue later on. This function will
#     set up the decoder and dataset information all into one Dataset class so that you can avoid the brute work later on.
#     Your file_pattern is very important in locating the files later. 

#     INPUTS:
#     - split_name(str): 'train' or 'validation'. Used to get the correct data split of tfrecord files
#     - dataset_dir(str): the dataset directory where the tfrecord files are located
#     - file_pattern(str): the file name structure of the tfrecord files in order to get the correct data
#     - file_pattern_for_counting(str): the string name to identify your tfrecord files for counting

#     OUTPUTS:
#     - dataset (Dataset): A Dataset class object where we can read its various components for easier batch creation later.
#     '''

    #First check whether the split_name is train or validation
    if split_name not in ['train', 'validation']:
        raise ValueError('The split_name %s is not recognized. Please input either train or validation as the split_name' % (split_name))

    #Create the full path for a general file_pattern to locate the tfrecord_files
    file_pattern_path = os.path.join(dataset_dir, file_pattern % (split_name))

    #Create a reader, which must be a TFRecord reader in this case
    reader = tf.TFRecordReader

    #Create the keys_to_features dictionary for the decoder
    keys_to_features = {
      'image/encoded': tf.FixedLenFeature((), tf.string, default_value=''),
      'image/id': tf.FixedLenFeature((), tf.int64, default_value=tf.zeros([], dtype=tf.int64)),
      'image/format': tf.FixedLenFeature((), tf.string, default_value='jpg'),
      'image/class/label': tf.FixedLenFeature(
          [], tf.int64, default_value=tf.zeros([], dtype=tf.int64)),
    }

    #Create the items_to_handlers dictionary for the decoder.
    items_to_handlers = {
    'image': slim.tfexample_decoder.Image(),
    'image_id': slim.tfexample_decoder.Tensor('image/id'),
    'label': slim.tfexample_decoder.Tensor('image/class/label'),
    }

    #Start to create the decoder
    decoder = slim.tfexample_decoder.TFExampleDecoder(keys_to_features, items_to_handlers)

    #Create the labels_to_name file
    labels_to_name_dict = labels_to_name

    #Actually create the dataset
    dataset = slim.dataset.Dataset(
        data_sources = file_pattern_path,
        decoder = decoder,
        reader = reader,
        #num_readers = 4,
        num_samples = 15678, #Number of training data
        num_classes = num_classes,
        labels_to_name = labels_to_name_dict,
        items_to_descriptions = items_to_descriptions)

    return dataset


Next step: Loading the training batches

In [7]:
def load_batch(dataset, batch_size, height=image_size, width=image_size, is_training=True):
    '''
    Loads a batch for training.

    INPUTS:
    - dataset(Dataset): a Dataset class object that is created from the get_split function
    - batch_size(int): determines how big of a batch to train
    - height(int): the height of the image to resize to during preprocessing
    - width(int): the width of the image to resize to during preprocessing
    - is_training(bool): to determine whether to perform a training or evaluation preprocessing

    OUTPUTS:
    - images(Tensor): a Tensor of the shape (batch_size, height, width, channels) that contain one batch of images
    - labels(Tensor): the batch's labels with the shape (batch_size,) (requires one_hot_encoding).

    '''
    #First create the data_provider object
    data_provider = slim.dataset_data_provider.DatasetDataProvider(
        dataset,
        common_queue_capacity = 24 + 3 * batch_size,
        common_queue_min = 24)

    #Obtain the raw image using the get method
    raw_image, image_id, label = data_provider.get(['image', 'image_id','label'])

    #Perform the correct preprocessing for this image depending if it is training or evaluating
    image = inception_daan_preprocessing.preprocess_image(raw_image, height, width, is_training)

    #As for the raw images, we just do a simple reshape to batch it up
    raw_image = tf.expand_dims(raw_image, 0)
    raw_image = tf.image.resize_nearest_neighbor(raw_image, [height, width])
    raw_image = tf.squeeze(raw_image)

    #Batch up the image by enqueing the tensors internally in a FIFO queue and dequeueing many elements with tf.train.batch.
    images, raw_images, image_ids, labels = tf.train.batch(
        [image, raw_image, image_id, label],
        batch_size = batch_size,
        num_threads = 4,
        capacity = 4 * batch_size,
        allow_smaller_final_batch = True)

    return images, raw_images, image_ids, labels


### RUNNING THE COMPLETE MODEL CODE BELOW

### FIRST: TRAINING THE MODEL

In [1]:
import csv

#================ DATASET INFORMATION ======================
#State dataset directory where the tfrecord files are located
dataset_dir = 'C:/Users\studentid\Desktop\JADS - Master Thesis\Data\ImagesPaddedLabel275'

#State where your log file is at. If it doesn't exist, create it.
log_dir = 'C:/Users\studentid\Desktop\JADS - Master Thesis\Data\ImagesPaddedLabel275/TRAIN-FINAL'

#State where your checkpoint file is
checkpoint_file = 'C:/Users\studentid\Desktop\JADS - Master Thesis\Data/inception_resnet_v2_2016_08_30.ckpt'

#State the image size you're resizing your images to. We will use the default inception size of 299.
image_size = 299

#State the number of classes to predict:
num_classes = 275 

#State the labels file and read it
labels_file = 'C:/Users\studentid\Desktop\JADS - Master Thesis\Data\ImagesFinalData/labels.txt'
labels = open(labels_file, 'r')

#Create a dictionary to refer each label to their string name
labels_to_name = {}
for line in labels:
    label, string_name = line.split(':')
    string_name = string_name[:-1] #Remove newline
    labels_to_name[int(label)] = string_name

#Create the file pattern of your TFRecord files so that it could be recognized later on
file_pattern = 'productimages_%s_*.tfrecord'

#Create a dictionary that will help people understand your dataset better. This is required by the Dataset class later.
items_to_descriptions = {
    'image': 'Image of a product in the dataset, out of 275 classes',
    'image_id': 'Unique identifier for each image',
    'label': 'A label that matches the class of the image.'
}


#================= TRAINING INFORMATION ==================
#State the number of epochs to train
num_epochs = 40 #After a while, model is converged, and stops improving.

#State your batch size
batch_size = 13 

base_lr = 0.0001
max_lr = 0.005
stepsize = 1206 #to find optimal values (stepsize x nr epochs = stepsize) (Known as: Learning Rate Range Test)
#Normal training: stepsize = 3x num_iterations ; Thus then stepsize is 3618.

#============== DATASET LOADING ======================
#We now create a function that creates a Dataset class which will give us many TFRecord files to feed in the examples into a queue in parallel.
def get_split(split_name, dataset_dir, file_pattern=file_pattern):
    '''
    Obtains the split - training or validation - to create a Dataset class for feeding the examples into a queue later on. This function will
    set up the decoder and dataset information all into one Dataset class so that you can avoid the brute work later on.
    Your file_pattern is very important in locating the files later. 

    INPUTS:
    - split_name(str): 'train' or 'validation'. Used to get the correct data split of tfrecord files
    - dataset_dir(str): the dataset directory where the tfrecord files are located
    - file_pattern(str): the file name structure of the tfrecord files in order to get the correct data
    - file_pattern_for_counting(str): the string name to identify your tfrecord files for counting

    OUTPUTS:
    - dataset (Dataset): A Dataset class object where we can read its various components for easier batch creation later.
    '''

    #First check whether the split_name is train or validation
    if split_name not in ['train', 'validation']:
        raise ValueError('The split_name %s is not recognized. Please input either train or validation as the split_name' % (split_name))

    #Create the full path for a general file_pattern to locate the tfrecord_files
    file_pattern_path = os.path.join(dataset_dir, file_pattern % (split_name))

    #Create a reader, which must be a TFRecord reader in this case
    #reader = tf.TFRecordReader
    reader = tf.TFRecordReader

    #Create the keys_to_features dictionary for the decoder
    keys_to_features = {
      'image/encoded': tf.FixedLenFeature((), tf.string, default_value=''),
      'image/id': tf.FixedLenFeature((), tf.int64, default_value=tf.zeros([], dtype=tf.int64)),
      'image/format': tf.FixedLenFeature((), tf.string, default_value='jpg'),
      'image/class/label': tf.FixedLenFeature(
          [], tf.int64, default_value=tf.zeros([], dtype=tf.int64)),
    }

    #Create the items_to_handlers dictionary for the decoder.
    items_to_handlers = {
    'image': slim.tfexample_decoder.Image(),
    'image_id': slim.tfexample_decoder.Tensor('image/id'),
    'label': slim.tfexample_decoder.Tensor('image/class/label'),
    }

    #Start to create the decoder
    decoder = slim.tfexample_decoder.TFExampleDecoder(keys_to_features, items_to_handlers)

    #Create the labels_to_name file
    labels_to_name_dict = labels_to_name

    #Actually create the dataset
    dataset = slim.dataset.Dataset(
        data_sources = file_pattern_path,
        decoder = decoder,
        reader = reader,
        num_readers = 4,
        num_samples = 16068, #15678
        num_classes = num_classes,
        labels_to_name = labels_to_name_dict,
        items_to_descriptions = items_to_descriptions)

    return dataset


def load_batch(dataset, batch_size, height=image_size, width=image_size, is_training=True):
    '''
    Loads a batch for training.

    INPUTS:
    - dataset(Dataset): a Dataset class object that is created from the get_split function
    - batch_size(int): determines how big of a batch to train
    - height(int): the height of the image to resize to during preprocessing
    - width(int): the width of the image to resize to during preprocessing
    - is_training(bool): to determine whether to perform a training or evaluation preprocessing

    OUTPUTS:
    - images(Tensor): a Tensor of the shape (batch_size, height, width, channels) that contain one batch of images
    - labels(Tensor): the batch's labels with the shape (batch_size,) (requires one_hot_encoding).

    '''
    #First create the data_provider object
    data_provider = slim.dataset_data_provider.DatasetDataProvider(
        dataset,
        common_queue_capacity = 24 + 3 * batch_size,
        common_queue_min = 24)

    #Obtain the raw image using the get method
    raw_image, label, image_id = data_provider.get(['image', 'label', 'image_id'])

    #Perform the correct preprocessing for this image depending if it is training or evaluating
    image = inception_daan_preprocessing.preprocess_image(raw_image, height, width, is_training) #Edited preprocessing

    #As for the raw images, we just do a simple reshape to batch it up
    raw_image = tf.expand_dims(raw_image, 0)
    raw_image = tf.image.resize_nearest_neighbor(raw_image, [height, width])
    raw_image = tf.squeeze(raw_image)

    #Batch up the image by enqueing the tensors internally in a FIFO queue and dequeueing many elements with tf.train.batch.
    images, raw_images, labels, image_ids = tf.train.batch(
        [image, raw_image, label, image_id],
        batch_size = batch_size,
        num_threads = 4,
        capacity = 4 * batch_size,
        allow_smaller_final_batch = True)

    return images, raw_images, labels, image_ids

def run():
    
    # Create lists for accuracy vs learning_rate plot
    acc_list = []
    lr_list = []
    
    #Create the log directory here. Must be done here otherwise import will activate this unneededly.
    if not os.path.exists(log_dir):
        os.mkdir(log_dir)  


    #======================= TRAINING PROCESS =========================
    #Now we start to construct the graph and build our model
    with tf.Graph().as_default() as graph:
        tf.logging.set_verbosity(tf.logging.INFO) #Set the verbosity to INFO level
        
        #First create the dataset and load one batch
        dataset = get_split('train', dataset_dir, file_pattern=file_pattern)
        images, _, labels, image_ids = load_batch(dataset, batch_size=batch_size)

        #Know the number steps to take before decaying the learning rate and batches per epoch
        num_batches_per_epoch = math.ceil(dataset.num_samples / batch_size) ## Used to be int(dataset...../batch..)
        num_steps_per_epoch = num_batches_per_epoch #Because one step is one batch processed

        #Create the model inference
        with slim.arg_scope(inception_resnet_v2_arg_scope()):
            logits, end_points = inception_resnet_v2(images, num_classes = dataset.num_classes, is_training = True)

        #Define the scopes that you want to exclude for restoration
        exclude = ['InceptionResnetV2/Logits', 'InceptionResnetV2/AuxLogits']
        variables_to_restore = slim.get_variables_to_restore(exclude = exclude)

        #Perform one-hot-encoding of the labels (Try one-hot-encoding within the load_batch function!)
        one_hot_labels = slim.one_hot_encoding(labels, dataset.num_classes)

        #Performs the equivalent to tf.nn.sparse_softmax_cross_entropy_with_logits but enhanced with checks
        loss = tf.losses.softmax_cross_entropy(onehot_labels = one_hot_labels, logits = logits)
        total_loss = tf.losses.get_total_loss()    #obtain the regularization losses as well

        #Create the global step for monitoring the learning_rate and training.
        global_step = get_or_create_global_step()
        
        
        #Created an implementation of Cyclic LR, instead of exponential decay
        lr1 = learning_rate_decay_daan.cyclic_lr(0.001, global_step, stepsize, base_lr, max_lr)

        #Define your exponentially decaying learning rate #This is the old learning_rate model.
#         lr = tf.train.exponential_decay(
#             learning_rate = initial_learning_rate,
#             global_step = global_step,
#             decay_steps = decay_steps,
#             decay_rate = learning_rate_decay_factor,
#             staircase = True)


        optimizer = tf.train.AdamOptimizer(learning_rate = lr1)

        #Create the train_op.
        train_op = slim.learning.create_train_op(total_loss, optimizer)

        #State the metrics that you want to predict. We get a predictions that is not one_hot_encoded.
        predictions = tf.argmax(end_points['Predictions'], 1)
        probabilities = end_points['Predictions']
        accuracy, accuracy_update = tf.contrib.metrics.streaming_accuracy(predictions, labels)
        metrics_op = tf.group(accuracy_update, probabilities)
        

        #Now finally create all the summaries you need to monitor and group them into one summary op.
        tf.summary.scalar('losses/Total_Loss', total_loss)
        tf.summary.scalar('accuracy', accuracy)
        tf.summary.scalar('learning_rate', lr1)
        my_summary_op = tf.summary.merge_all()

        #Now we need to create a training step function that runs both the train_op, metrics_op and updates the global_step concurrently.
        def train_step(sess, train_op, global_step):
            '''
            Simply runs a session for the three arguments provided and gives a logging on the time elapsed for each global step
            '''
            #Check the time for each sess run
            start_time = time.time()
            total_loss, global_step_count, _ = sess.run([train_op, global_step, metrics_op])
            time_elapsed = time.time() - start_time

            #Run the logging to print some results
            logging.info('global step %s: loss: %.4f (%.2f sec/step)', global_step_count, total_loss, time_elapsed)

            return total_loss, global_step_count

        #Now we create a saver function that actually restores the variables from a checkpoint file in a sess
        saver = tf.train.Saver(variables_to_restore)
        def restore_fn(sess):
            return saver.restore(sess, checkpoint_file)

        #Define your supervisor for running a managed session. Do not run the summary_op automatically or else it will consume too much memory
        sv = tf.train.Supervisor(logdir = log_dir, summary_op = None, init_fn = restore_fn)
        
        
        def plotlist(sess, acc, lr):
        
            acc_value, lr_value = sess.run([acc, lr])

            return acc_value, lr_value
            
        
        #Run the managed session
        with sv.managed_session() as sess:

            for step in range(num_steps_per_epoch * num_epochs):
                #At the start of every epoch, show the vital information:
                if step % num_batches_per_epoch == 0:
                    logging.info('Epoch %s/%s', step/num_batches_per_epoch + 1, num_epochs)
                    learning_rate_value, accuracy_value = sess.run([lr1, accuracy])
                    logging.info('Current Learning Rate: %s', learning_rate_value)
                    logging.info('Current Streaming Accuracy: %s', accuracy_value)

                    # optionally, print your logits and predictions for a sanity check that things are going fine.
                    logits_value, probabilities_value, predictions_value, labels_value = sess.run([logits, probabilities, predictions, labels])
                    print('logits: \n'), logits_value
                    print('Probabilities: \n'), probabilities_value
                    print('predictions: \n'), predictions_value
                    print('Labels:\n:'), labels_value

                #Store accuracy value & learning rate value every step during the training.
                #This is for LR Range Test (see below)
                
                #Log the summaries every 10 step.
                if step % 10 == 0:
                    loss, _ = train_step(sess, train_op, sv.global_step)
                    summaries = sess.run(my_summary_op)
                    sv.summary_computed(sess, summaries)
                    accstore, lrstore = plotlist(sess, accuracy, lr1)
                    acc_list.append(accstore)
                    lr_list.append(lrstore)
                
#                 if step == 10:
#                     break
                    
                #If not, simply run the training step
                else:
                    loss, _ = train_step(sess, train_op, sv.global_step)
                    accstore, lrstore = plotlist(sess, accuracy, lr1)
                    acc_list.append(accstore)
                    lr_list.append(lrstore)
                    
                
            
            #We log the final training loss and accuracy
            logging.info('Final Loss: %s', loss)
            logging.info('Final Accuracy: %s', sess.run(accuracy))
            
            #Just to be sure, write the acc_list and lr_list to a csv, to also get back to these in normal python scripts.
            outfile_acc = open('LR-rangetest-acc.csv','w')
            out_acc = csv.writer(outfile_acc)
            out_acc.writerows(map(lambda x: [x], acc_list))
            outfile_acc.close()
            
            outfile_lr = open('LR-rangetest-lr.csv','w')
            out_lr = csv.writer(outfile_lr)
            out_lr.writerows(map(lambda x: [x], lr_list))
            outfile_lr.close()
            
            fig3, ax3 = plt.subplots()
            ax3.plot(lr_list, acc_list, color='b', linestyle ='solid')
            ax3.autoscale(enable=True, axis="y", tight=False)
            
            plt.xlabel('Learning Rate')
            plt.ylabel('Accuracy Value')
            
            plt.show()

            #Once all the training has been done, save the log files and checkpoint model
            logging.info('Finished training! Saving model to disk now.')
            sv.saver.save(sess, sv.save_path, global_step = sv.global_step)

            
if __name__ == '__main__':
    run()


## Test the model on new data ; EVALUATION

In [ ]:
import tensorflow as tf
from tensorflow.python.platform import tf_logging as logging
from tensorflow.contrib.framework.python.ops.variables import get_or_create_global_step
import inception_preprocessing
from inception_resnet_v2 import inception_resnet_v2, inception_resnet_v2_arg_scope
import time
import os
#from train_flowers import get_split, load_batch
import matplotlib.pyplot as plt
plt.style.use('ggplot')
slim = tf.contrib.slim
import csv

#State your log directory where you can retrieve your model
log_dir = 'C:/Users\s164677\Desktop\JADS - Master Thesis\Data\ImagesPaddedLabel\log1'

#Create a new evaluation log directory to visualize the validation process
log_eval = 'C:/Users\s164677\Desktop\JADS - Master Thesis\Data\ImagesPaddedLabel/log_eval_test'

#State the dataset directory where the validation set is found
dataset_dir = 'C:/Users\s164677\Desktop\JADS - Master Thesis\Data\ImagesPaddedLabel'

#State the batch_size to evaluate each time, which can be a lot more than the training batch
batch_size = 26

#State the number of classes
num_classes = 275

#State the number of epochs to evaluate
num_epochs = 10 #Model stops improving after +- 3 epochs.

#Get the latest checkpoint file
checkpoint_file = tf.train.latest_checkpoint(log_dir)

#============== DATASET LOADING ======================
#We now create a function that creates a Dataset class which will give us many TFRecord files to feed in the examples into a queue in parallel.
def get_split(split_name, dataset_dir, file_pattern=file_pattern):
    '''
    Obtains the split - training or validation - to create a Dataset class for feeding the examples into a queue later on. This function will
    set up the decoder and dataset information all into one Dataset class so that you can avoid the brute work later on.
    Your file_pattern is very important in locating the files later. 

    INPUTS:
    - split_name(str): 'train' or 'validation'. Used to get the correct data split of tfrecord files
    - dataset_dir(str): the dataset directory where the tfrecord files are located
    - file_pattern(str): the file name structure of the tfrecord files in order to get the correct data
    - file_pattern_for_counting(str): the string name to identify your tfrecord files for counting

    OUTPUTS:
    - dataset (Dataset): A Dataset class object where we can read its various components for easier batch creation later.
    '''

    #First check whether the split_name is train or validation
    if split_name not in ['train', 'validation']:
        raise ValueError('The split_name %s is not recognized. Please input either train or validation as the split_name' % (split_name))

    #Create the full path for a general file_pattern to locate the tfrecord_files
    file_pattern_path = os.path.join(dataset_dir, file_pattern % (split_name))

    #Create a reader, which must be a TFRecord reader in this case
    reader = tf.TFRecordReader

    #Create the keys_to_features dictionary for the decoder
    keys_to_features = {
      'image/encoded': tf.FixedLenFeature((), tf.string, default_value=''),
      'image/id': tf.FixedLenFeature((), tf.int64, default_value=tf.zeros([], dtype=tf.int64)),
      'image/format': tf.FixedLenFeature((), tf.string, default_value='jpg'),
      'image/class/label': tf.FixedLenFeature(
          [], tf.int64, default_value=tf.zeros([], dtype=tf.int64)),
    }

    #Create the items_to_handlers dictionary for the decoder.
    items_to_handlers = {
    'image': slim.tfexample_decoder.Image(),
    'image_id': slim.tfexample_decoder.Tensor('image/id'),
    'label': slim.tfexample_decoder.Tensor('image/class/label'),
    }

    #Start to create the decoder
    decoder = slim.tfexample_decoder.TFExampleDecoder(keys_to_features, items_to_handlers)

    #Create the labels_to_name file
    labels_to_name_dict = labels_to_name

    #Actually create the dataset
    dataset = slim.dataset.Dataset(
        data_sources = file_pattern_path,
        decoder = decoder,
        reader = reader,
        num_readers = 4,
        shuffle=False,
        num_samples = 5344, #Amount of training dataset (2), validation dataset (2) number also goes here.
        num_classes = num_classes,
        labels_to_name = labels_to_name_dict,
        items_to_descriptions = items_to_descriptions)

    return dataset


def load_batch(dataset, batch_size, height=image_size, width=image_size, is_training=False):
    '''
    Loads a batch for training.

    INPUTS:
    - dataset(Dataset): a Dataset class object that is created from the get_split function
    - batch_size(int): determines how big of a batch to train
    - height(int): the height of the image to resize to during preprocessing
    - width(int): the width of the image to resize to during preprocessing
    - is_training(bool): to determine whether to perform a training or evaluation preprocessing

    OUTPUTS:
    - images(Tensor): a Tensor of the shape (batch_size, height, width, channels) that contain one batch of images
    - labels(Tensor): the batch's labels with the shape (batch_size,) (requires one_hot_encoding).

    '''
    #First create the data_provider object
    data_provider = slim.dataset_data_provider.DatasetDataProvider(
        dataset, shuffle=False,
        common_queue_capacity = 24 + 3 * batch_size,
        common_queue_min = 24)

    #Obtain the raw image using the get method
    raw_image, label, image_id = data_provider.get(['image', 'label', 'image_id'])

    #Perform the correct preprocessing for this image depending if it is training or evaluating
    image = inception_daan_preprocessing.preprocess_image(raw_image, height, width, is_training)

    #As for the raw images, we just do a simple reshape to batch it up
    raw_image = tf.expand_dims(raw_image, 0)
    raw_image = tf.image.resize_nearest_neighbor(raw_image, [height, width])
    raw_image = tf.squeeze(raw_image)

    #Batch up the image by enqueing the tensors internally in a FIFO queue and dequeueing many elements with tf.train.batch.
    images, raw_images, labels, image_ids = tf.train.batch(
        [image, raw_image, label, image_id],
        batch_size = batch_size,
        num_threads = 1, #To prevent shuffling the output, which makes it unable to use ensemble later on. Threads = 1.
        capacity = 4 * batch_size,
        allow_smaller_final_batch = True)

    return images, raw_images, labels, image_ids


def run():
    
    probabilities_list = []
    label_list = []
    pred_list = []
    id_list = []
    #Create log_dir for evaluation information
    if not os.path.exists(log_eval):
        os.mkdir(log_eval)

    #Just construct the graph from scratch again
    with tf.Graph().as_default() as graph:
        tf.logging.set_verbosity(tf.logging.INFO)
        #Get the dataset first and load one batch of validation images and labels tensors. Set is_training as False so as to use the evaluation preprocessing
        dataset = get_split('validation', dataset_dir)
        images, raw_images, labels, image_ids = load_batch(dataset, batch_size = batch_size, is_training = False)

        #Create some information about the training steps
        num_batches_per_epoch = (dataset.num_samples / batch_size)
        num_steps_per_epoch = num_batches_per_epoch
        print('steps per epoch: %s' %(num_steps_per_epoch))
        print('number of test images: %s' %(dataset.num_samples))
        

        #Now create the inference model but set is_training=False
        with slim.arg_scope(inception_resnet_v2_arg_scope()):
            logits, end_points = inception_resnet_v2(images, num_classes = dataset.num_classes, is_training = False)

        # #get all the variables to restore from the checkpoint file and create the saver function to restore
        variables_to_restore = slim.get_variables_to_restore()
        saver = tf.train.Saver(variables_to_restore)
        def restore_fn(sess):
            return saver.restore(sess, checkpoint_file)

        #Just define the metrics to track without the loss or whatsoever
        predictions = tf.argmax(end_points['Predictions'], 1)
        
        
        probabilities = end_points['Predictions']
        
        accuracy, accuracy_update = tf.contrib.metrics.streaming_accuracy(predictions, labels)
        metrics_op = tf.group(accuracy_update)

        #Create the global step and an increment op for monitoring
        global_step = get_or_create_global_step()
        global_step_op = tf.assign(global_step, global_step + 1) #no apply_gradient method so manually increasing the global_step
        

        #Create a evaluation step function
        def eval_step(sess, metrics_op, global_step):
#             '''
#             Simply takes in a session, runs the metrics op and some logging information.
#             '''
            start_time = time.time()
            _, global_step_count, accuracy_value = sess.run([metrics_op, global_step_op, accuracy])
            time_elapsed = time.time() - start_time

            #Log some information
            logging.info('Global Step %s: Streaming Accuracy: %.4f (%.2f sec/step)', global_step_count, accuracy_value, time_elapsed)

            return accuracy_value

        
        def store_lists(sess, probabilities, image_ids, labels, predictions):
            prob_batch, ids_batch, labels_batch, pred_batch = sess.run([probabilities, image_ids, labels, predictions])
            
            return prob_batch, ids_batch, labels_batch, pred_batch
        

        #Define some scalar quantities to monitor
        tf.summary.scalar('Validation_Accuracy', accuracy)
        my_summary_op = tf.summary.merge_all()

        #Get your supervisor
        sv = tf.train.Supervisor(logdir = log_eval, summary_op = None, saver = None, init_fn = restore_fn)

        #Now we are ready to run in one session
        with sv.managed_session() as sess:
            for step in range(int((num_steps_per_epoch * num_epochs))):  ### HAD TO MAKE INT OF IT, GAVE FLOAT ERROR               
                sess.run(sv.global_step)
                             
                #Get all necessary values to append to lists
                prob_store, id_store, label_store, pred_store = store_lists(sess, probabilities, image_ids, labels, predictions)
                probabilities_list.append(prob_store)
                id_list.append(id_store)
                label_list.append(label_store)
                pred_list.append(pred_store)
                
                #print vital information every start of the epoch as always
                if step % num_batches_per_epoch == 0:
                    logging.info('Epoch: %s/%s', step / num_batches_per_epoch + 1, num_epochs)
                    logging.info('Current Streaming Accuracy: %.4f', sess.run(accuracy))
                    
                   
                    
                #Compute summaries every 10 steps and continue evaluating
                if step % 10 == 0:
                    eval_step(sess, metrics_op = metrics_op, global_step = sv.global_step)
                    summaries = sess.run(my_summary_op)
                    sv.summary_computed(sess, summaries)
                    
                    #Get all necessary values to append to lists
                    prob_store, id_store, label_store, pred_store = store_lists(sess, probabilities, image_ids, labels, predictions)
                    probabilities_list.append(prob_store)
                    id_list.append(id_store)
                    label_list.append(label_store)
                    pred_list.append(pred_store)


                #Otherwise just run as per normal
                else:
                    eval_step(sess, metrics_op = metrics_op, global_step = sv.global_step)
                    
                    #Get all necessary values to append to lists
                    prob_store, id_store, label_store, pred_store = store_lists(sess, probabilities, image_ids, labels, predictions)
                    probabilities_list.append(prob_store)
                    id_list.append(id_store)
                    label_list.append(label_store)
                    pred_list.append(pred_store)
    
            
            #At the end of all the evaluation, show the final accuracy
            logging.info('Final Streaming Accuracy: %.4f', sess.run(accuracy)) 
            
         
        
            #And write these to a csv
            id_list = np.asarray(id_list) ## ID is necessary, to map all probabilities to correct images!
            #Because IRNV2 swaps the batches, the order is then shuffled. So a unique identifier is necessary.
            with open('output-imageid.csv', 'w', newline='') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerows(id_list)
            
            probabilities_list = np.asarray(probabilities_list) #The predicted probabilities
            with open('output-probabilities.csv', 'w', newline='') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerows(probabilities_list)
            
            label_list = np.asarray(label_list) #The correct labels
            with open('output-labels.csv', 'w', newline='') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerows(label_list)
            
            pred_list = np.asarray(pred_list) #The predicted labels
            with open ('output-predictions.csv', 'w', newline='') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerows(pred_list)
            

            #Now we want to visualize the last batch's images just to see what our model has predicted
            raw_images, labels, predictions = sess.run([raw_images, labels, predictions])
            for i in range(10):
                image, label, prediction = raw_images[i], labels[i], predictions[i]
                prediction_name, label_name = dataset.labels_to_name[prediction], dataset.labels_to_name[label]
                text = 'Prediction: %s \n Ground Truth: %s' %(prediction_name, label_name)
                img_plot = plt.imshow(image)

                #Set up the plot and hide axes
                plt.title(text)
                img_plot.axes.get_yaxis().set_ticks([])
                img_plot.axes.get_xaxis().set_ticks([])
                plt.show()

            logging.info('Model evaluation has completed! Visit TensorBoard for more information regarding your evaluation.')

if __name__ == '__main__':
    run()
